In [12]:
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from detect_outlier import *
from sklearn.model_selection import train_test_split


In [13]:
df = read_data_no_outlier(0)

In [14]:
df_normalized =df.drop("address", axis= 1).drop("createdAt", axis= 1).drop("zscore", axis= 1)
main_label_column = df['main_label']
sub_label_column = df['sub_label']
df_normalized = (df_normalized - df_normalized.min()) / (df_normalized.max() - df_normalized.min())
df_normalized = df_normalized * 550 + 300
df_normalized['main_label'] = main_label_column
df_normalized['sub_label'] = sub_label_column

In [15]:
def score(label):
    if label == 0:
        return np.random.randint(300, 580)
    if label == 1:
        return np.random.randint(580, 670)
    if label == 2:
        return np.random.randint(670, 740)
    if label == 3:
        return np.random.randint(740, 800)
    if label == 4:
        return np.random.randint(800, 850)


def labeling(scores):
    label = []
    for score in scores:
        if score < 580:
            label.append(0)
        elif score >= 580 and score < 670:
            label.append(1)
        elif score >= 670 and score < 740:
            label.append(2)
        elif score >= 740 and score < 800:
            label.append(3)
        elif score >= 800:
            label.append(4)
    return np.array(label)

In [16]:
X = np.array(df_normalized[
    [
        "balanceInUSD",
        # "borrowInUSD",
        # "depositInUSD",
        "averageTotalAsset",
        "frequencyMountOfTransaction",
        "borrow_per_balance",
        "deposit_per_balance",
        "borrow_per_deposit",
        "totalValueOfLiquidation",
        "numberOfLiquidation",
        "frequencyOfTransaction",
        "frequencyOfDappTransactions",
        "numberOfInteractedDapps",
        "typesOfInteractedDapps",
        "numberOfReputableDapps",
        "age",
    ]
].values)
y = df_normalized["main_label"].values
new_y = np.array([score(y[i]) for i in range(len(y))])

X_train, X_test, new_y_train, new_y_test, y_train, y_test  = train_test_split(
            X, new_y, y, test_size=0.2, random_state=42
        )
# Chuẩn bị dữ liệu


In [17]:
X_train.shape[0]

16535

In [18]:
# a= [-1.21308685  4.47542451 -5.51931091  3.71804369  2.0969602   4.80669811
#  -2.03392303  7.7603754   4.16328747 -4.93222656  4.23313938 -8.20674732
#  -6.96695354 -0.1264665 ]

In [19]:
y_test[1]

2.0

In [20]:
import numpy as np

def new_error_func(y, sub_y, y_pred):
    errors = [0] * len(y_pred)
    # Lặp qua từng phần tử trong mảng
    for i in range(len(y_pred)):
        if y[i] > sub_y[i]:
            errors[i] = y_pred[i] - y[i] 
        else:
            errors[i] = y_pred[i] - sub_y[i] 
    
    return errors

# Hàm tính giá trị của hàm lỗi (MSE)
def mean_squared_error(X, y, w):
    n = len(y)
    y_pred = labeling(np.dot(X, w))
    mse = np.sum((y - y_pred) ** 2) / n
    return mse

# Hàm tính gradient của hàm lỗi (MSE)
def gradient_mean_squared_error(X, y, w):
    n = len(y)
    y_pred = labeling(np.dot(X, w))
    
    error = y_pred - y
    gradient = 2 * np.dot(X.T, error) / n
    return gradient

# Gradient descent để tối ưu hóa hàm lỗi (MSE)
def stochastic_gradient_descent(X, y, learning_rate=0.01, num_iterations=10000, decay_rate=0.95, tol=1e-5):
    w = np.random.uniform(-10, 10, X.shape[1])
    prev_loss = float('inf')
    for i in range(num_iterations):
        random_index = np.random.randint(0, len(X))
        X_sample = X[random_index:random_index+1]
        y_sample = y[random_index:random_index+1]
        
        grad = gradient_mean_squared_error(X_sample, y_sample, w)
        
        w -= learning_rate * grad
        learning_rate *= decay_rate  # Decay learning rate
        
        if i % (num_iterations/10) == 0:
            loss = mean_squared_error(X, y, w)
            print(f"Iteration {i}: Loss = {loss}")
    return w

# Áp dụng gradient descent để tối ưu hóa hàm lỗi (MSE)
learned_weights = stochastic_gradient_descent(X_train, y_train)

print("Vector trọng số tối ưu:", learned_weights)


Iteration 0: Loss = 4.116722104626549
Iteration 1000: Loss = 2.2719685515573027
Iteration 2000: Loss = 2.2719685515573027
Iteration 3000: Loss = 2.2719685515573027
Iteration 4000: Loss = 2.2719685515573027
Iteration 5000: Loss = 2.2719685515573027
Iteration 6000: Loss = 2.2719685515573027
Iteration 7000: Loss = 2.2719685515573027
Iteration 8000: Loss = 2.2719685515573027
Iteration 9000: Loss = 2.2719685515573027
Vector trọng số tối ưu: [-1.0271625   8.45193282 -0.04746718  7.53956989 -7.27798467 -8.02956583
 -9.414798    4.55514751 -5.13683007  4.70694817 -1.87240563  2.91499178
  0.99902865  3.84368889]


In [21]:
pred = labeling(X_test.dot(learned_weights))
print(accuracy_score(y_test, pred))

0.30672472181906146


In [22]:
import numpy as np

unique_elements, counts = np.unique(pred, return_counts=True)

for element, count in zip(unique_elements, counts):
    print("Phần tử", element, "xuất hiện", count, "lần")


Phần tử 0 xuất hiện 707 lần
Phần tử 1 xuất hiện 506 lần
Phần tử 2 xuất hiện 978 lần
Phần tử 3 xuất hiện 523 lần
Phần tử 4 xuất hiện 1420 lần
